# Networking

Networking is comprised of layers. The lower levels are unaware of what’s happening above them, and higher levels are agnostic to what’s happening below them. Lower levels receive a stream of bytes and pass it on. Higher levels don’t care how messages are sent; they just want them sent.

## HTTP

HTTP is an application-level protocol. Its job is to transport content like HTML, CSS, JavaScript, WebAssembly modules, images, video, and other formats. These formats often include other embedded formats via compression and encoding standards. HTTP itself often redundantly includes information provided by one of the layers below it, TCP. Between HTTP and TCP sits TLS. TLS (Transport Layer Security), which has replaced SSL (Secure Sockets Layer), adds the S to HTTPS.

TLS provides encrypted messaging over an unencrypted connection. TLS is implemented on top of TCP. TCP sits upon many other protocols. These go all the way down to specifying how voltages should be interpreted as 0s and 1s.

HTTP can make adjustments downwards to TCP. The `Connection: keep-alive` HTTP header instructs TCP to maintain its connection after this HTTP message has been received. These sorts of interactions occur all through the stack.


- HTTP has rules for telling each of the parties when the connection ends.
- Data is transmitted as a byte stream. Rules for translating the message body from `[u8]` to String (or perhaps an image, video, or some other content) are handled as part of the protocol. This can be tedious to handle manually as HTTP allows content to be compressed into several methods and encoded into several plain text formats.
- HTTP defaults to port 80.

## TCP

Rust’s standard library provides us with cross-platform tools for making TCP requests.

In [14]:
use std::io::prelude::*;
use std::net::TcpStream;

let host = "www.rustinaction.com:80"; // explicitly specifying the port (80) is required. TcpStream does not know that this will become a HTTP request.
let mut conn = TcpStream::connect(host)?;

conn.write_all(b"GET / HTTP/1.0");
conn.write_all(b"\r\n")?; // \r\n signifies a new line in many networking protocols

conn.write_all(b"Host: www.rustinaction.com")?;
conn.write_all(b"\r\n\r\n")?; // two blank new lines signify end of request

std::io::copy(&mut conn, &mut std::io::stdout())?; // std::io::copy() streams bytes from a Reader to a Writer.

HTTP/1.0 301 Moved Permanently
Content-Type: text/html; charset=utf-8
Location: https://www.rustinaction.com/
Permissions-Policy: interest-cohort=()
Vary: Origin
Date: Fri, 24 Mar 2023 11:27:31 GMT
Content-Length: 64

<a href="https://www.rustinaction.com/">Moved Permanently</a>.



We are using HTTP 1.0 to ensure that the connection is closed when the server sends its response. HTTP 1.0 does not support “keep alive” requests. Specifying HTTP 1.1 actually confuses this code as the server will refuse to close the connection until it has received another request, and the client will never send one.

We have added `Host: www.rustinaction.com` because the connection is established over IP, which does not have host names. When `TcpStream::connect()` connects to the server, it only uses an IP address. Adding the Host HTTP header allows us to inject that information back into the context.

### What Is A Port Number

Port numbers are purely virtual. They are simply u16 values. Port numbers allow a single IP address to host multiple services.

## Domain Name Resolution

To send messages over the internet, the IP (internet protocol) address is required. TCP knows nothing about domain names. To convert a domain name to an IP address, we rely on the Domain Name System (DNS) and its process called domain name resolution.

We’re able to resolve names by asking a server, which can recursively ask other servers. DNS requests can be made over TCP, including encryption with TLS, but are also sent over UDP (User Datagram Protocol). 

There are some public DNS providers. Their IP addresses are:
- 1.1.1.1 and 1.0.0.1 by Cloudflare
- 8.8.8.8 and 8.8.4.4. by Google
- 9.9.9.9 by Quad9 (founded by IBM)
- 64.6.64.6 and 64.6.65.6 by VeriSign

## How DNS supports connections within UDP

UDP does not have a notion of long-lived connections. Unlike TCP, all messages are short-lived and one-way. Put another way, UDP does not support two-way (duplex) communications. But DNS requires a response to be sent from the DNS server back to the client.

To enable two-way communications within UDP, both parties must act as clients and servers, depending on context. That context is defined by the protocol built on top of UDP. Within DNS, the client becomes a DNS server to receive the server’s reply. The following table provides a flow chart of the process.

| Stage                        | DNS client role | DNS server role |
| ---------------------------- | --------------- | --------------- |
| Request sent from DNS client |    UDP client   |   UDP server    |
| Reply sent from DNS server   |    UDP server   |   UDP client    |